In [21]:
import json, time, os, sys, glob
import csv
import shutil
import warnings
import numpy as np
import torch
from torch import optim
from torch.utils.data import DataLoader
from dateutil import parser
import queue
import copy
import torch.nn as nn
import torch.nn.functional as F
import random
import os.path
import subprocess
from concurrent.futures import ProcessPoolExecutor    
from utils_Baker import worker_init_fn, get_pdbs, loader_pdb, PDB_dataset, StructureDataset, StructureLoader
from model_utils_Baker import featurize, loss_smoothed, loss_nll, get_std_opt, ProteinMPNN

In [5]:
model_list = sorted(glob.glob('../Models/class_0/model/model_weights/*'))
model_list
for j in reversed(range(0, len(model_list), 10)):
    print(model_list[j])

../Models/class_0/model/model_weights/epoch_last.pt
../Models/class_0/model/model_weights/epoch305_step247599.pt
../Models/class_0/model/model_weights/epoch255_step239849.pt
../Models/class_0/model/model_weights/epoch230_step235734.pt
../Models/class_0/model/model_weights/epoch205_step231934.pt
../Models/class_0/model/model_weights/epoch180_step228134.pt
../Models/class_0/model/model_weights/epoch155_step224334.pt


In [6]:
model_list

['../Models/class_0/model/model_weights/epoch155_step224334.pt',
 '../Models/class_0/model/model_weights/epoch155_step224349.pt',
 '../Models/class_0/model/model_weights/epoch160_step225094.pt',
 '../Models/class_0/model/model_weights/epoch160_step225124.pt',
 '../Models/class_0/model/model_weights/epoch165_step225854.pt',
 '../Models/class_0/model/model_weights/epoch165_step225899.pt',
 '../Models/class_0/model/model_weights/epoch170_step226614.pt',
 '../Models/class_0/model/model_weights/epoch170_step226674.pt',
 '../Models/class_0/model/model_weights/epoch175_step227374.pt',
 '../Models/class_0/model/model_weights/epoch175_step227449.pt',
 '../Models/class_0/model/model_weights/epoch180_step228134.pt',
 '../Models/class_0/model/model_weights/epoch180_step228224.pt',
 '../Models/class_0/model/model_weights/epoch185_step228894.pt',
 '../Models/class_0/model/model_weights/epoch185_step228999.pt',
 '../Models/class_0/model/model_weights/epoch190_step229654.pt',
 '../Models/class_0/model

In [22]:
# This function is pulled from utils_Baker.py but modified for testing
def build_training_clusters(params):
    test_ids = set([int(l) for l in open(params['TEST']).readlines()])

    # read & clean list.csv
    with open(params['LIST'], 'r') as f:
        reader = csv.reader(f)
        next(reader)
        rows = [[r[0],r[3],int(r[4])] for r in reader
                if float(r[2])<=params['RESCUT'] and
                parser.parse(r[1])<=parser.parse(params['DATCUT'])]
    
    ## we want to remove proteins that are listed in list.csv that aren't actually in the dataset
    files = glob.glob('../../training/ProteinMPNN_training_data//pdb_2021aug02/pdb/*/*.pt')
    files = [x.split('/')[-1].split('.pt')[0] for x in files]
    files = set(files) # set for faster membership checking

    # compile test sets
    test = {}
    for r in rows:
        if r[0] in files:
            if r[2] in test_ids:
                if r[2] in test.keys():
                    test[r[2]].append(r[:2])
                else:
                    test[r[2]] = [r[:2]]
     
    return test


In [23]:
classes = ['class_0','class_1','class_2','class_3','class_4','class_0_1_2','class_3_4']
model_list = glob.glob('../Models/'+classes[0]+'/model/model_weights/*')
print(model_list)

device = torch.device("cuda:1" if (torch.cuda.is_available()) else "cpu")
print(device)


['../Models/class_0/model/model_weights/epoch215_step233649.pt', '../Models/class_0/model/model_weights/epoch185_step228999.pt', '../Models/class_0/model/model_weights/epoch260_step240624.pt', '../Models/class_0/model/model_weights/epoch160_step225124.pt', '../Models/class_0/model/model_weights/epoch155_step224334.pt', '../Models/class_0/model/model_weights/epoch175_step227449.pt', '../Models/class_0/model/model_weights/epoch240_step237254.pt', '../Models/class_0/model/model_weights/epoch295_step246049.pt', '../Models/class_0/model/model_weights/epoch215_step233454.pt', '../Models/class_0/model/model_weights/epoch200_step231324.pt', '../Models/class_0/model/model_weights/epoch340_step253024.pt', '../Models/class_0/model/model_weights/epoch335_step252249.pt', '../Models/class_0/model/model_weights/epoch225_step235199.pt', '../Models/class_0/model/model_weights/epoch280_step243724.pt', '../Models/class_0/model/model_weights/epoch195_step230414.pt', '../Models/class_0/model/model_weights/

In [24]:
PATH = model_list[0]
logfile = '../Models/'+classes[0]+'/model/BakerLab_test_log.txt'

data_path = '../../training/ProteinMPNN_training_data//pdb_2021aug02'
params = {
    "LIST"    : f"{data_path}/list.csv",
    "TEST"    : f"{data_path}/test_clusters.txt",
    "DIR"     : f"{data_path}",
    "DATCUT"  : "2030-Jan-01",
    "RESCUT"  : 3.5, #resolution cutoff for PDBs
    "HOMO"    : 0.70 #min seq.id. to detect homo chains
    }

LOAD_PARAM = {'batch_size': 1,
                'shuffle': True,
                'pin_memory':False,
                'num_workers': 4,
                }

test = build_training_clusters(params)
test_set = PDB_dataset(list(test.keys()), loader_pdb, test, params)
test_loader = torch.utils.data.DataLoader(test_set, worker_init_fn=worker_init_fn, **LOAD_PARAM)

# Once the test data is loaded, we will iterate through the models
model = ProteinMPNN(node_features=128, 
                    edge_features=128, 
                    hidden_dim=128, 
                    num_encoder_layers=3, 
                    num_decoder_layers=3, 
                    k_neighbors=48, 
                    dropout=0.1, 
                    augment_eps=0.2)
model.to(device)

checkpoint = torch.load(PATH,map_location=torch.device('cpu'))
total_step = checkpoint['step'] #write total_step from the checkpoint
epoch = checkpoint['epoch'] #write epoch from the checkpoint
model.load_state_dict(checkpoint['model_state_dict'])


<All keys matched successfully>

In [25]:
with ProcessPoolExecutor(max_workers=12) as executor:
    # Load test dataset
    pdb_dict_test = get_pdbs(test_loader, 1, 10000, 1000000)
    dataset_test = StructureDataset(pdb_dict_test, truncate=None, max_length=10000)
    loader_test = StructureLoader(dataset_test, batch_size=10000)

    model.eval()
    with torch.no_grad():
        test_sum, test_weights = 0., 0.
        test_acc = 0.
        for _, batch in enumerate(loader_test):
            X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)
            log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all)
            mask_for_loss = mask * chain_M
            loss, loss_av, true_false = loss_nll(S, log_probs, mask_for_loss)

            test_sum += torch.sum(loss * mask_for_loss).cpu().data.numpy()
            test_acc += torch.sum(true_false * mask_for_loss).cpu().data.numpy()
            test_weights += torch.sum(mask_for_loss).cpu().data.numpy()

    test_loss = test_sum / test_weights
    test_accuracy = test_acc / test_weights
    test_perplexity = np.exp(test_loss)

    test_perplexity_ = np.format_float_positional(np.float32(test_perplexity), unique=False, precision=3)
    test_accuracy_ = np.format_float_positional(np.float32(test_accuracy), unique=False, precision=3)
    #with open(logfile, 'a') as f:
    #        f.write(f'epoch: {epoch}, test: {test_perplexity_}, test_acc: {test_accuracy_}\n')
    print(f'epoch: {epoch}, test: {test_perplexity_}, test_acc: {test_accuracy_}\n')

2duy_A mask tensor([0, 1], dtype=torch.int32)
2duy_A idx tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1])
______________
2uyt_A mask tensor([0], dtype=torch.int32)
2uyt_A idx tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

/stor/home/pwoolley/software/miniconda3/envs/mlfold/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/stor/home/pwoolley/software/miniconda3/envs/mlfold/lib/python3.11/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
def main(args):
    device = torch.device("cuda:1" if (torch.cuda.is_available()) else "cpu")
    print(device)

    PATH = args.previous_checkpoint

    logfile = '../Models/'+classes[0]+'/model/BakerLab_test_log.txt'

    data_path = args.path_for_test_data
    params = {
        "LIST"    : f"{data_path}/list.csv",
        "TEST"    : f"{data_path}/test_clusters.txt",
        "DIR"     : f"{data_path}",
        "DATCUT"  : "2030-Jan-01",
        "RESCUT"  : 3.5, #resolution cutoff for PDBs
        "HOMO"    : 0.70 #min seq.id. to detect homo chains
    }

    LOAD_PARAM = {'batch_size': 1,
                  'shuffle': True,
                  'pin_memory':False,
                  'num_workers': 4,
                  }

    test = build_training_clusters(params)
    
    test_set = PDB_dataset(list(test.keys()), loader_pdb, test, params)
    test_loader = torch.utils.data.DataLoader(test_set, worker_init_fn=worker_init_fn, **LOAD_PARAM)

    # Once the test data is loaded, we will iterate through the models
    model = ProteinMPNN(node_features=args.hidden_dim, 
                        edge_features=args.hidden_dim, 
                        hidden_dim=args.hidden_dim, 
                        num_encoder_layers=args.num_encoder_layers, 
                        num_decoder_layers=args.num_encoder_layers, 
                        k_neighbors=args.num_neighbors, 
                        dropout=args.dropout, 
                        augment_eps=args.backbone_noise)
    model.to(device)

    checkpoint = torch.load(PATH)
    total_step = checkpoint['step'] #write total_step from the checkpoint
    epoch = checkpoint['epoch'] #write epoch from the checkpoint
    model.load_state_dict(checkpoint['model_state_dict'])

    with ProcessPoolExecutor(max_workers=12) as executor:
        # Load test dataset
        pdb_dict_test = get_pdbs(test_loader, 1, args.max_protein_length, args.num_examples_per_epoch)
        dataset_test = StructureDataset(pdb_dict_test, truncate=None, max_length=args.max_protein_length)
        loader_test = StructureLoader(dataset_test, batch_size=args.batch_size)

        model.eval()
        with torch.no_grad():
            test_sum, test_weights = 0., 0.
            test_acc = 0.
            for _, batch in enumerate(loader_test):
                X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)
                log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all)
                mask_for_loss = mask * chain_M
                loss, loss_av, true_false = loss_nll(S, log_probs, mask_for_loss)

                test_sum += torch.sum(loss * mask_for_loss).cpu().data.numpy()
                test_acc += torch.sum(true_false * mask_for_loss).cpu().data.numpy()
                test_weights += torch.sum(mask_for_loss).cpu().data.numpy()

        test_loss = test_sum / test_weights
        test_accuracy = test_acc / test_weights
        test_perplexity = np.exp(test_loss)

        test_perplexity_ = np.format_float_positional(np.float32(test_perplexity), unique=False, precision=3)
        test_accuracy_ = np.format_float_positional(np.float32(test_accuracy), unique=False, precision=3)
        with open(logfile, 'a') as f:
                f.write(f'epoch: {epoch}, test: {test_perplexity_}, test_acc: {test_accuracy_}\n')
        print(f'epoch: {epoch}, test: {test_perplexity_}, test_acc: {test_accuracy_}\n')

if __name__ == "__main__":
    argparser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)

    argparser.add_argument("--path_for_test_data", type=str, default="my_path/pdb_test", help="path for loading test data")
    argparser.add_argument("--path_for_outputs", type=str, default="./exp_020", help="path for logs and model weights")
    argparser.add_argument("--batch_size", type=int, default=10000, help="number of tokens for one batch")
    argparser.add_argument("--max_protein_length", type=int, default=10000, help="maximum length of the protein complex")
    argparser.add_argument("--hidden_dim", type=int, default=128, help="hidden model dimension")
    argparser.add_argument("--num_encoder_layers", type=int, default=3, help="number of encoder layers")
    argparser.add_argument("--num_decoder_layers", type=int, default=3, help="number of decoder layers")
    argparser.add_argument("--num_neighbors", type=int, default=48, help="number of neighbors for the sparse graph")
    argparser.add_argument("--dropout", type=float, default=0.1, help="dropout level; 0.0 means no dropout")
    argparser.add_argument("--rescut", type=float, default=3.5, help="PDB resolution cutoff")
    argparser.add_argument("--debug", type=bool, default=False, help="minimal data loading for debugging")

    args = argparser.parse_args()
    main(args)


In [ ]:
import argparse
import os.path

def main(args):
    import json, time, os, sys, glob
    import shutil
    import warnings
    import numpy as np
    import torch
    from torch import optim
    from torch.utils.data import DataLoader
    import queue
    import copy
    import torch.nn as nn
    import torch.nn.functional as F
    import random
    import os.path
    import subprocess
    from concurrent.futures import ProcessPoolExecutor    
    from utils import worker_init_fn, get_pdbs, loader_pdb, build_training_clusters, PDB_dataset, StructureDataset, StructureLoader
    from model_utils import featurize, loss_smoothed, loss_nll, get_std_opt, ProteinMPNN

    scaler = torch.cuda.amp.GradScaler()
     
    device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

    base_folder = time.strftime(args.path_for_outputs, time.localtime())

    if base_folder[-1] != '/':
        base_folder += '/'
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    subfolders = ['model_weights']
    for subfolder in subfolders:
        if not os.path.exists(base_folder + subfolder):
            os.makedirs(base_folder + subfolder)

    PATH = args.previous_checkpoint

    logfile = base_folder + 'log.txt'
    if not PATH:
        with open(logfile, 'w') as f:
            f.write('Epoch\tTrain\tValidation\n')

    data_path = args.path_for_training_data
    params = {
        "LIST"    : f"{data_path}/list.csv", 
        "VAL"     : f"{data_path}/valid_clusters.txt",
        "TEST"    : f"{data_path}/test_clusters.txt",
        "DIR"     : f"{data_path}",
        "DATCUT"  : "2030-Jan-01",
        "RESCUT"  : args.rescut, #resolution cutoff for PDBs
        "HOMO"    : 0.70 #min seq.id. to detect homo chains
    }


    LOAD_PARAM = {'batch_size': 1,
                  'shuffle': True,
                  'pin_memory':False,
                  'num_workers': 4}


    if args.debug:
        args.num_examples_per_epoch = 50
        args.max_protein_length = 1000
        args.batch_size = 1000

    train, valid, test = build_training_clusters(params, args.debug)
    
    train_set = PDB_dataset(list(train.keys()), loader_pdb, train, params)
    train_loader = torch.utils.data.DataLoader(train_set, worker_init_fn=worker_init_fn, **LOAD_PARAM)
    valid_set = PDB_dataset(list(valid.keys()), loader_pdb, valid, params)
    valid_loader = torch.utils.data.DataLoader(valid_set, worker_init_fn=worker_init_fn, **LOAD_PARAM)
    


    model = ProteinMPNN(node_features=args.hidden_dim, 
                        edge_features=args.hidden_dim, 
                        hidden_dim=args.hidden_dim, 
                        num_encoder_layers=args.num_encoder_layers, 
                        num_decoder_layers=args.num_encoder_layers, 
                        k_neighbors=args.num_neighbors, 
                        dropout=args.dropout, 
                        augment_eps=args.backbone_noise)
    model.to(device)


    if PATH:
        checkpoint = torch.load(PATH)
        total_step = checkpoint['step'] #write total_step from the checkpoint
        epoch = checkpoint['epoch'] #write epoch from the checkpoint
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        total_step = 0
        epoch = 0

    optimizer = get_std_opt(model.parameters(), args.hidden_dim, total_step)


    if PATH:
        optimizer.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


    with ProcessPoolExecutor(max_workers=12) as executor:
        q = queue.Queue(maxsize=3)
        p = queue.Queue(maxsize=3)
        for i in range(3):
            q.put_nowait(executor.submit(get_pdbs, train_loader, 1, args.max_protein_length, args.num_examples_per_epoch))
            p.put_nowait(executor.submit(get_pdbs, valid_loader, 1, args.max_protein_length, args.num_examples_per_epoch))
        pdb_dict_train = q.get().result()
        pdb_dict_valid = p.get().result()
        
        print(pdb_dict_train,pdb_dict_valid)

        dataset_train = StructureDataset(pdb_dict_train, truncate=None, max_length=args.max_protein_length) 
        dataset_valid = StructureDataset(pdb_dict_valid, truncate=None, max_length=args.max_protein_length)
        
        loader_train = StructureLoader(dataset_train, batch_size=args.batch_size)
        loader_valid = StructureLoader(dataset_valid, batch_size=args.batch_size)

        reload_c = 0 
        for e in range(args.num_epochs):
            t0 = time.time()
            e = epoch + e
            model.train()
            train_sum, train_weights = 0., 0.
            train_acc = 0.
            if e % args.reload_data_every_n_epochs == 0:
                if reload_c != 0:
                    pdb_dict_train = q.get().result()
                    dataset_train = StructureDataset(pdb_dict_train, truncate=None, max_length=args.max_protein_length)
                    loader_train = StructureLoader(dataset_train, batch_size=args.batch_size)
                    pdb_dict_valid = p.get().result()
                    dataset_valid = StructureDataset(pdb_dict_valid, truncate=None, max_length=args.max_protein_length)
                    loader_valid = StructureLoader(dataset_valid, batch_size=args.batch_size)
                    q.put_nowait(executor.submit(get_pdbs, train_loader, 1, args.max_protein_length, args.num_examples_per_epoch))
                    p.put_nowait(executor.submit(get_pdbs, valid_loader, 1, args.max_protein_length, args.num_examples_per_epoch))
                reload_c += 1
            for _, batch in enumerate(loader_train):
                start_batch = time.time()
                X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)
                elapsed_featurize = time.time() - start_batch
                optimizer.zero_grad()
                mask_for_loss = mask*chain_M

                if args.mixed_precision:
                    with torch.cuda.amp.autocast():
                        log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all)
                        _, loss_av_smoothed = loss_smoothed(S, log_probs, mask_for_loss)
           
                    scaler.scale(loss_av_smoothed).backward()
                     
                    if args.gradient_norm > 0.0:
                        total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), args.gradient_norm)

                    scaler.step(optimizer)
                    scaler.update()
                else:
                    log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all)
                    _, loss_av_smoothed = loss_smoothed(S, log_probs, mask_for_loss)
                    loss_av_smoothed.backward()

                    if args.gradient_norm > 0.0:
                        total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), args.gradient_norm)

                    optimizer.step()
                
                loss, loss_av, true_false = loss_nll(S, log_probs, mask_for_loss)
            
                train_sum += torch.sum(loss * mask_for_loss).cpu().data.numpy()
                train_acc += torch.sum(true_false * mask_for_loss).cpu().data.numpy()
                train_weights += torch.sum(mask_for_loss).cpu().data.numpy()

                total_step += 1

            model.eval()
            with torch.no_grad():
                validation_sum, validation_weights = 0., 0.
                validation_acc = 0.
                for _, batch in enumerate(loader_valid):
                    X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)
                    log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all)
                    mask_for_loss = mask*chain_M
                    loss, loss_av, true_false = loss_nll(S, log_probs, mask_for_loss)

                    validation_sum += torch.sum(loss * mask_for_loss).cpu().data.numpy()
                    validation_acc += torch.sum(true_false * mask_for_loss).cpu().data.numpy()
                    validation_weights += torch.sum(mask_for_loss).cpu().data.numpy()
    
            train_loss = train_sum / train_weights
            train_accuracy = train_acc / train_weights
            train_perplexity = np.exp(train_loss)
            validation_loss = validation_sum / validation_weights
            validation_accuracy = validation_acc / validation_weights
            validation_perplexity = np.exp(validation_loss)
            
            train_perplexity_ = np.format_float_positional(np.float32(train_perplexity), unique=False, precision=3)     
            validation_perplexity_ = np.format_float_positional(np.float32(validation_perplexity), unique=False, precision=3)
            train_accuracy_ = np.format_float_positional(np.float32(train_accuracy), unique=False, precision=3)
            validation_accuracy_ = np.format_float_positional(np.float32(validation_accuracy), unique=False, precision=3)
    
            t1 = time.time()
            dt = np.format_float_positional(np.float32(t1-t0), unique=False, precision=1) 
            with open(logfile, 'a') as f:
                f.write(f'epoch: {e+1}, step: {total_step}, time: {dt}, train: {train_perplexity_}, valid: {validation_perplexity_}, train_acc: {train_accuracy_}, valid_acc: {validation_accuracy_}\n')
            print(f'epoch: {e+1}, step: {total_step}, time: {dt}, train: {train_perplexity_}, valid: {validation_perplexity_}, train_acc: {train_accuracy_}, valid_acc: {validation_accuracy_}')
            
